In [1]:
from tools.pipeline.config import ONEDRIVE_PATH
import os, json
from tqdm import tqdm
from tools.core.llm_query import LLMQuery

llm = LLMQuery()

## 옳지 않은 문제 중 해설 많은 애들 선별하기

In [ ]:
# import os, json
# # from tools.pipeline.config import ONEDRIVE_PATH
# ONEDRIVE_PATH = 'C:\\Users\\Jin\\Onedrive\\데이터L\\selectstar'

# classified = json.load(open(os.path.join(ONEDRIVE_PATH, 'evaluation', 'eval_data', '7_multiple_rw', 'answer_type_classified.json')))
# wrong_data = [p for p in classified if p['answer_type'] == 'wrong']
# len(wrong_data)

In [ ]:
# from tools.core.llm_query import LLMQuery

# llm_query = LLMQuery()

# def is_full_explanation(llm, question, answer, options, explanation):
#     system_prompt = """다음은 문제, 정답, 해설입니다. 해설이 문제의 모든 선지에 대한 설명을 포함하는지 확인해주세요.
#     만약 모두 포함하면 'full'을 반환하고, 포함하지 않으면 'notfull'을 반환해주세요.
#     """
#     user_prompt = f"""
#     문제: {question}
#     정답: {answer}
#     선지: {options}
#     해설: {explanation}
#     """
#     response = llm.query_openrouter(system_prompt, user_prompt, model_name='google/gemini-2.5-flash')
#     return response

In [ ]:
# wrong_data[0]

In [ ]:
# response = is_full_explanation(llm_query, wrong_data[0]['question'], wrong_data[0]['answer'], wrong_data[0]['options'], wrong_data[0]['explanation'])
# print(response)

In [ ]:
# from tqdm import tqdm

# full_explanation = []
# notfull_explanation = []
# fail = []

# for wd in tqdm(wrong_data):
#     response = is_full_explanation(llm_query, wd['question'], wd['answer'], wd['options'], wd['explanation'])
#     if response == 'full':
#         full_explanation.append(wd)
#     elif response == 'notfull':
#         notfull_explanation.append(wd)
#     else:
#         fail.append(wd)

# len(full_explanation)+len(notfull_explanation)+len(fail) == len(wrong_data)

In [ ]:
# with open(os.path.join(ONEDRIVE_PATH, 'evaluation/eval_data/7_multiple_rw/full_explanation.json'), 'w', encoding='utf-8') as f:
#     json.dump(full_explanation, f, ensure_ascii=False, indent=4)

# with open(os.path.join(ONEDRIVE_PATH, 'evaluation/eval_data/7_multiple_rw/notfull_explanation.json'), 'w', encoding='utf-8') as f:
#     json.dump(notfull_explanation, f, ensure_ascii=False, indent=4)

# with open(os.path.join(ONEDRIVE_PATH, 'evaluation/eval_data/7_multiple_rw/fail.json'), 'w', encoding='utf-8') as f:
#     json.dump(fail, f, ensure_ascii=False, indent=4)

# 키워드 뽑고 서술형 문제로 바꾸기

In [2]:
full_explanation = json.load(open(os.path.join(ONEDRIVE_PATH, 'evaluation', 'eval_data', '9_multiple_to_essay', 'full_explanation.json')))
len(full_explanation)

859

In [3]:
system_prompt = """
다음 지시문을 따르세요. 당신의 역할은 주어진 객관식 항목(선지)과 해설만을 근거로 문제의 핵심 주제를 추론하고, 이를 서술형 문제로 변환하는 것입니다.

입력으로 제공되는 것
- 선택지: 문자열 배열
- 정답: 정답 표기(예: '①' 또는 인덱스)
- 해설: 정답에 관한 설명 또는 정오 수정 설명

필수 규칙
1) 자료 제한: 질문지와 선택지, 해설만 사용하세요. 최대한 원 질문을 보존하세요.
2) 주제 추출: "~에 관해 적절하지 않은 것은?"의 앞부분을 주제로 도출하세요.
3) 서술형 변환: “[주제]에 관해 서술하시오.” 형태로 만들되, 앞에 “다음 키워드를 활용하여”를 붙여 한 문장으로 제시하세요.
4) 선지 분류: 해설을 근거로 틀린 선지 1개와 옳은 선지 3~4개를 식별하세요. 옳은 선지가 3개 미만이면 가능한 만큼만 사용하세요.
   - 정답 표기가 있는 경우, 해설과의 정합성을 우선으로 판단하여 틀린 선지를 확정하세요.
5) 키워드 추출: 틀린 선지 1개와 옳은 선지 3~4개 각각에서 핵심 개념을 1개씩 뽑아 키워드를 구성하세요. 총 5~10개 이내로 하며, 다음 원칙을 지키세요.
   - 중복 키워드는 제거하고, 명사구 중심으로 간결히 표현하세요.
6) 출력은 정확히 두 줄만 생성하세요.
   - 1행: 서술형 문제: 다음 키워드를 활용하여 [주제]에 관해 서술하시오.
   - 2행: 키워드: 키워드1, 키워드2, 키워드3, …

검증 체크리스트
- 키워드는 10개 이내인가?
- 키워드가 주제를 적절하게 반영하고 있는가?
- 주제 문구가 명확하고 간결한가?

예시 입력 형식
선택지: ['① ...', '② ...', '③ ...', '④ ...']
정답: '①'
해설: '...'

예시 출력 형식
서술형 문제: 다음 키워드를 활용하여 [도출한 주제]에 관해 서술하시오.
키워드: [키워드A], [키워드B], [키워드C], ...
"""

In [4]:
for q in tqdm(full_explanation):
    user_prompt = f"""
========= 문제 ========
- 문제 지문: {q['question']}
- 선지: {q['options']}
- 정답: {q['answer']}
- 해설: {q['explanation']}
"""
    response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'google/gemini-2.5-flash')
    q['essay_question'], q['essay_keyword'] = response.strip().split('\n')
    q['essay_question'] = q['essay_question'].replace('서술형 문제: ', '')
    q['essay_keyword'] = q['essay_keyword'].replace('키워드: ', '')

100%|██████████| 859/859 [25:24<00:00,  1.78s/it]


In [ ]:
with open(os.path.join(ONEDRIVE_PATH, 'evaluation', 'eval_data', '9_multiple_to_essay', 'essay_w_keyword.json'), 'w', encoding='utf-8') as f:
    json.dump(full_explanation, f, ensure_ascii=False, indent=4)

In [6]:
full_explanation[0]

{'file_id': 'SS03015',
 'title': 'SD에듀 보험계리사 1차 한권으로 끝내기 5권 (기출문제해설)',
 'chapter': '문제편',
 'tag': 'q_0026_0003',
 'domain': '경제',
 'subdomain': '미시경제학',
 'classification_reason': '독점적 경쟁시장 특징과 장기균형, 시장이론 핵심',
 'is_calculation': False,
 'question': '독점적 경쟁시장에 관한 설명으로 옳지 않은 것은?',
 'options': ['① 수많은 공급자가 가격 수용자로 행동한다.',
  '② 장기균형에서 기업들의 경제학적 이윤은 0이다.',
  '③ 장기균형에서는 평균비용곡선의 최저점보다 더 적은 양을 생산한다.',
  '④ 수많은 공급자가 서로 차별화된 상품을 공급하지만, 공급된 상품들의 대체성이 높다.'],
 'answer': '①',
 'explanation': '독점적 경쟁시장에서 공급자는 가격 수용자가 아니라 가격 설정자로서 행동한다.\n② 독점적 경쟁시장에 기업들은 정상이윤만 획득하기에 경제학적 이윤은 0이 된다.\n③ 독점적 경쟁시장에서 장기균형조건은 P=SAC=LAC>MR=SMC=LMC 이므로 생산량은 평균비용곡선의 최저점보다 적은 양을 생산한다.\n④ 차별화된 상품들이 재화에 근본적인 차이가 날 정도로 차이가 존재하지 않아 공급된 상품들의 대체성이 높다.',
 'answer_type': 'wrong',
 'essay_question': '다음 키워드를 활용하여 독점적 경쟁시장에 관해 서술하시오.',
 'essay_keyword': '가격 설정자, 경제학적 이윤 0, 평균비용곡선 최저점, 차별화된 상품, 높은 대체성'}